# 🦜 LangChain `create_agent`: High-Level Agent Development - Interactive Demo

**Welcome to the high-level agent development demonstration!**

In the previous notebook, we built a graph manually (defining nodes, edges, and state) using LangGraph. That approach gave us complete control but required substantial setup and boilerplate code.

This notebook introduces **`create_agent`**, a high-level function from the `langchain` module that dramatically simplifies agent creation.

## 💡 What is `create_agent`?

`create_agent` is a **pre-assembled LangGraph** - a production-ready agent template that handles all the common patterns automatically.

Under the hood, `create_agent` automatically:
1. Creates a `StateGraph` with proper state management
2. Adds a model node for reasoning (LLM calls)
3. Adds a tool node for executing actions
4. Connects them with the correct edges and loops
5. Compiles it into a ready-to-use `CompiledGraph`

**The result?** You get the same powerful agent capabilities with ~80% less code.

## 📚 What This Demo Covers

### **1. `create_agent` Function**
Instantiating a pre-built ReAct (Reasoning + Acting) agent workflow without manually defining nodes and edges.

### **2. Standard Interrupts**
Handling missing information by pausing agent execution (same pattern as before, but with less code).

### **3. Middleware Integration**
A powerful way to add cross-cutting concerns to your agent - like human approval steps, logging, monitoring, or validation - without modifying the core agent logic.

---

## 🎯 When to Use `create_agent` vs Manual Graphs

**Use `create_agent` when:**
- You need a standard ReAct agent (reason → act → reason → act loop)
- You want rapid prototyping and development
- Your use case fits the common agent pattern
- You prefer less boilerplate code

**Build manual graphs when:**
- You need custom workflow logic beyond ReAct
- You require specialized routing or conditional logic
- You're implementing novel agent architectures
- You need fine-grained control over every step

---

**This demo shows you:**
- How to create production-ready agents in just a few lines
- The abstraction layers that LangChain provides
- Adding middleware for approval workflows
- Understanding what's happening under the hood

In [1]:
from langgraph.prebuilt import create_react_agent
from langraph_prompts import ACTION_PROMPT_TEMPLATE
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command, interrupt
from datetime import datetime

In [2]:
# Diagnostic cell: prints kernel info and tests imports
import sys, traceback, os
print('Notebook kernel executable:', sys.executable)
print('Python version:', sys.version.replace('\n',' '))
print('Working directory:', os.getcwd())

# Test core package
try:
    import langchain_core
    print('langchain_core installed at:', getattr(langchain_core, '__file__', str(langchain_core)))
except Exception:
    print('Failed to import langchain_core:' )
    traceback.print_exc()

# Test specific imports used in this notebook
try:
    from langgraph.prebuilt import create_react_agent
    print('create_react_agent import: OK')
except Exception:
    print('Failed to import create_react_agent:')
    traceback.print_exc()

try:
    import langraph_prompts
    print('langraph_prompts loaded from:', getattr(langraph_prompts, '__file__', str(langraph_prompts)))
except Exception:
    print('Failed to import langraph_prompts:')
    traceback.print_exc()
    

Notebook kernel executable: /opt/anaconda3/bin/python
Python version: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ]
Working directory: /Users/massadraza/Desktop/Langchain Prep/langchain-langgraph-demo
langchain_core installed at: /opt/anaconda3/lib/python3.13/site-packages/langchain_core/__init__.py
create_react_agent import: OK
langraph_prompts loaded from: /Users/massadraza/Desktop/Langchain Prep/langchain-langgraph-demo/langraph_prompts.py


In [ ]:
@tool
def get_time_off_balance(user_id: str) -> int:
    '''
    Get the time off balance for a user. 

    
    Args :
        user_id: A unique string representing the user id

    Returns:
        An integer value representing the time-off balance
    '''
    return 10

@tool
def process_time_off(user_id: str, start_date: str, end_date: str) -> dict:
    '''
    Process time-off request for a user

    Args :
        user_id: A unique string representing the user id
        start_date: A date string in YYYY-MM-DD format for the start date of the time-off request
        end_date: A date string in YYYY-MM-DD format for the end date of the time-off request

    Returns:
        A dictionary with the following keys:
        - status: A string value representing the status of the time-off request
        - message: A string value representing the message of the time-off request
    '''
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    days_requested = end_date - start_date
    time_off_balance = get_time_off_balance.invoke({"user_id": user_id})

    result = {}

    if days_requested.days + 1 > time_off_balance:
        result['status'] = 'error'
        result['message'] = f'Time off request failed. you only have {time_off_balance} days of remaining leaves but requested {days_requested.days} days'

    else:
        time_off_balance -= days_requested.days + 1
        result['status'] = 'success'
        result['message'] = f'Time off request processed successfully for {days_requested.days + 1} days. Your remaining time off balance is {time_off_balance}'


    return result


@tool
def get_additional_info_from_human( message: str) -> str:
    '''
        Raises an interrupt to fetch additional information from the human requesting the action

        Args:
            message: a message string from the AI requesting the user for missing information

        Returns:
            A string value representing the response with additional information from the human

    '''
    interrupt_result = interrupt(message)

    return interrupt_result['user_message']


## 1) Creating the Agent (The Easy Way)

* **Manual Graph (Previous Notebook):** Like building a PC from scratch. You buy the CPU, RAM, and Motherboard, and connect the wires yourself. It's powerful but takes time.
* **`create_agent` (This Notebook):** Like buying a pre-built laptop. You just turn it on. It has the same components inside (LLM, Tools, Memory), but LangChain assembled them for you.

###  Code Context
In the code below:
* `create_agent(...)`: This single function replaces the `StateGraph`, `add_node`, `add_edge`, and `compile` steps.
* It automatically creates a graph where the LLM decides to call tools and loops back until it's done.
* `checkpointer=InMemorySaver()`: We still need this to enable "Pause/Resume" functionality.

📚 **[Docs: create_agent](https://docs.langchain.com/oss/python/langchain/agents)**

In [10]:
user_id = '1' 
llm = init_chat_model(model="gpt-4o", temperature=0)
tools = [get_time_off_balance, process_time_off, get_additional_info_from_human]

todays_date = datetime.now().strftime("%Y-%m-%d")

system_prompt = PromptTemplate(template=ACTION_PROMPT_TEMPLATE).invoke({'todays_date': todays_date, 'user_id': user_id})

agent = create_react_agent(llm, tools=tools, prompt=system_prompt.text, checkpointer=InMemorySaver())

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [13]:
user_message = "I want to take leave"

config = {'configurable': {'thread_id': user_id}}

In [14]:
result = agent.invoke({"messages": [{"role": "user", "content": user_message}]}, config=config)

result

{'messages': [HumanMessage(content='I want to take leave', additional_kwargs={}, response_metadata={}, id='abf14e2b-38d2-4cd7-a5f3-732f0e081abf'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tuEALRPRgzzIRlt0spms29JN', 'function': {'arguments': '{"message":"Please provide the start and end dates for your leave request."}', 'name': 'get_additional_info_from_human'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 343, 'total_tokens': 373, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a0e9480a2f', 'id': 'chatcmpl-Cuu8xv8PddMGzJEMGi9D82iyjggqe', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d24c6af7-9edd-4a4b-81d6-ffc171f44f20-0', tool_cal

## 1.1) Resuming the Agent

Just like in the previous notebook, our `get_additional_info_from_human` tool triggered an **Interrupt**. The agent is now paused, waiting for the "End Date".
We use the same `Command(resume=...)` pattern to provide the answer and un-pause execution.

### Code Context
In the code below:
* `agent.invoke(...)`: We pass the `Command` object with the user's answer (`"tomorrow"`).
* The agent takes this answer, pretends the tool returned it, and continues its logic to book the leave.

In [15]:
user_message = "tomorrow"

result = agent.invoke(Command(resume={'user_message': user_message}), config=config)

result

{'messages': [HumanMessage(content='I want to take leave', additional_kwargs={}, response_metadata={}, id='abf14e2b-38d2-4cd7-a5f3-732f0e081abf'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tuEALRPRgzzIRlt0spms29JN', 'function': {'arguments': '{"message":"Please provide the start and end dates for your leave request."}', 'name': 'get_additional_info_from_human'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 343, 'total_tokens': 373, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a0e9480a2f', 'id': 'chatcmpl-Cuu8xv8PddMGzJEMGi9D82iyjggqe', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d24c6af7-9edd-4a4b-81d6-ffc171f44f20-0', tool_cal

## 3) Middleware (Human-in-the-Loop)

Middleware in LangChain is a new abstraction that lets you hook into and control the agent’s core loop at defined points (before the model is called, as the model is called, and after the model/tool step finishes). It is conceptually similar to HTTP middleware in web frameworks: you can stack multiple middleware components that each inspect or modify state as a request “flows” through the agent.

Each middleware can read and modify the agent state, messages, tools, or model configuration at specific hook points, without changing the agent implementation itself.

In our code, we are adding this "Are you sure?" popup to the **Book Leave** tool to prevent accidental bookings.

### Code Context
In the code below:
* `HumanInTheLoopMiddleware`: This is the component that intercepts tool calls.
* `interrupt_on`: This dictionary defines **which** tools trigger the pause. We map `process_time_off` to specific permissions, ensuring the agent cannot run this specific tool without external approval.

📚 **[Docs: Middleware](https://docs.langchain.com/oss/python/langchain/middleware/overview)**

In [17]:
# Using a custom approach for human-in-the-loop instead of middleware
# The create_react_agent already supports interrupts through the get_additional_info_from_human tool

agent_with_middleware = create_react_agent(
    llm, 
    tools=tools, 
    prompt=system_prompt.text, 
    checkpointer=InMemorySaver()
)

# Note: For tool-specific approval, you would implement this logic in the tool itself
# or use a custom node in a StateGraph (as shown in Demo 2)

In [19]:
# Using a custom approach for human-in-the-loop instead of middleware
# The create_react_agent already supports interrupts through the get_additional_info_from_human tool

agent_with_middleware = create_react_agent(
    llm, 
    tools=tools, 
    prompt=system_prompt.text, 
    checkpointer=InMemorySaver()
)

# Note: For tool-specific approval, you would implement this logic in the tool itself
# or use a custom node in a StateGraph (as shown in Demo 2)

In [20]:
user_message = "I want to take leave on 18-12-2025"
config = {'configurable': {'thread_id': user_id}}

In [21]:
agent_with_middleware.invoke({"messages": [{"role": "user", "content": user_message}]}, config=config)

{'messages': [HumanMessage(content='I want to take leave on 18-12-2025', additional_kwargs={}, response_metadata={}, id='752d9011-3aa0-4411-aee5-0629300d9af0'),
  AIMessage(content="To process your time-off request for December 18, 2025, I'll first need to check your current time-off balance. Let me do that for you.", additional_kwargs={'tool_calls': [{'id': 'call_2rIPBKbMhZs1uS7LhKdFfWs4', 'function': {'arguments': '{"user_id":"1"}', 'name': 'get_time_off_balance'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 351, 'total_tokens': 402, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a0e9480a2f', 'id': 'chatcmpl-CuuB9vPXp40omzlTb8Gt13lqV1BQ0', 'service_tier': 'default', 'finish_reason': 'tool_calls',

In [ ]:
agent_with_middleware.invoke(Command(resume={"decisions": [{"type": "approve"}]}), config=config)

{'messages': [HumanMessage(content='I want to take leave on 18-12-2025', additional_kwargs={}, response_metadata={}, id='752d9011-3aa0-4411-aee5-0629300d9af0'),
  AIMessage(content="To process your time-off request for December 18, 2025, I'll first need to check your current time-off balance. Let me do that for you.", additional_kwargs={'tool_calls': [{'id': 'call_2rIPBKbMhZs1uS7LhKdFfWs4', 'function': {'arguments': '{"user_id":"1"}', 'name': 'get_time_off_balance'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 351, 'total_tokens': 402, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a0e9480a2f', 'id': 'chatcmpl-CuuB9vPXp40omzlTb8Gt13lqV1BQ0', 'service_tier': 'default', 'finish_reason': 'tool_calls',